##Welcome to my NYC Airbnb EDA project - my second EDA project.##

We will validate the data to make sure the .csv file is appropriate to work with, and if not then we will clean the data and handle the issues that we might run into with Pandas.

After the validation we analyze the data for any underlying trends and interesting metrics that we want to investigate, and I will present these findings visually through: Matplotlib, Seaborn, and Tableau

In [10]:
import pandas as pd

# Here is where we will investigate and validate the data for completeness and correctness.
# I already did some initial exploration in Data Wrangler to see which columns had null values that needed to be replaced/filled.

filepath = '/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv'
df = pd.read_csv(filepath)

# Let's see what a row of the .csv looks like
print(df.head(1))

# Check for null values and fill 16 null values
print(df['name'].isnull().sum())
df['name'] = df['name'].fillna('Unknown')

# Check for null values and fill 21 null values
print(df['host_name'].isnull().sum())
df['host_name'] = df['host_name'].fillna('Unknown')

# Save the updated .csv for later work
df.to_csv('/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv', index=False)




     id                                name  host_id host_name  \
0  2539  Clean & quiet apt home by the park     2787      John   

  neighbourhood_group neighbourhood  latitude  longitude     room_type  price  \
0            Brooklyn    Kensington  40.64749  -73.97237  Private room    149   

   minimum_nights  number_of_reviews last_review  reviews_per_month  \
0               1                  9  2018-10-19               0.21   

   calculated_host_listings_count  availability_365  
0                               6               365  
16
21


Now that we've cleaned the data we can move onto analyzing and visualizing the data so that we can tell a story about Airbnbs in New York

We are going to break down our analysis and visualzation into three categories:

Beginner:

    1.	Which neighborhood group has the most Airbnb listings?
	2.	What is the average price of an Airbnb listing in each borough?
	3.	What is the distribution of room types across NYC?
	4.	Which 5 neighborhoods have the highest average price?
	5.	How many listings are available for at least 300 days a year?

Intermediate:

    1.	Is there a correlation between number of reviews and price?
	2.	What is the average price of listings based on room type and neighborhood group?
	3.	Which hosts have the most listings, and how much do they charge on average?
	4.	How does availability vary by borough across different room types?
	5.	What are the most reviewed listings in Manhattan and their price ranges?

Hard:

	1.	Can we identify clusters of high-priced listings using latitude and longitude (e.g., via KMeans)?
	2.	What is the price elasticity across room types and minimum night stays (does price predict duration)?
	3.	Is there a statistically significant price difference between boroughs (e.g., ANOVA)?
	4.	Can we build a model to predict listing price using features like room type, location, and availability?
	5.	How do availability and number of reviews impact price — controlling for neighborhood and room type (multivariate regression)?

Throughout my analysis I will answer these questions and will be able to compare the results with tableau as well.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned data
df = pd.read_csv('/Users/andrewhlavacek/Documents/EDA_NYCAirbnb/AB_NYC_2019.csv')

# Now we need to answer the BEGINNER questions.

# 1. Which neighborhood has the most listings?

# 2. What is the average price of listings in each neighborhood?

# 3.	What is the distribution of room types across NYC?

# 4.	Which 5 neighborhoods have the highest average price?

# 5.	How many listings are available for at least 300 days a year?